In [1]:
import findspark
findspark.init()
# Create SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Aşağıdaki ayarları bilgisayarınızın belleğine göre değiştirebilirsiniz
spark = SparkSession.builder \
.master("local[4]") \
.appName("Dataset-Olusturmak") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

# sparkContext'i kısaltmada tut
sc = spark.sparkContext

In [16]:
retailRDD2 = sc.textFile("C:\\Users\\toshiba\\SkyDrive\\veribilimi.co\\Datasets\\OnlineRetail.csv")

In [17]:
retailRDD = retailRDD2.filter(lambda x: 'InvoiceNo' not in x)

In [18]:
retailRDD.take(5)

['536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84029E;RED WOOLLY HOTTIE WHITE HEART.;6;1.12.2010 08:26;3,39;17850;United Kingdom']

In [22]:
def cancelled_price(line):
    is_cancelled = True if(line.split(";")[0].startswith("C")) else False
    quantity = float(line.split(";")[3])
    price = float(line.split(";")[5].replace(",","."))
    total = quantity * price
    
    return (is_cancelled, total)
    

In [23]:
cancelled_total = retailRDD.map(cancelled_price)

In [24]:
"""
2.5
3.50
1,4
,454
"""

'\n2.5\n3.50\n1,4\n,454\n'

In [25]:
cancelled_total.take(5)

[(False, 15.299999999999999),
 (False, 20.34),
 (False, 22.0),
 (False, 20.34),
 (False, 20.34)]

In [26]:
cancelled_total_reduce = cancelled_total.reduceByKey(lambda x,y: x+y)

In [27]:
cancelled_total_reduce.take(5)

[(False, 10644560.424000263), (True, -896812.4900000116)]

In [29]:
cancelled_total_reduce.filter(lambda x: x[0] == True).take(2)

[(True, -896812.4900000116)]

In [30]:
cancelled_total_reduce.filter(lambda x: x[0] == True).map(lambda x: x[1]).take(1)

[-896812.4900000116]